# 03 - Modelos de Forecasting

**Proyecto:** Forecast Promtur - Tráfico Orgánico  
**Objetivo:** Generar predicciones de métricas para 2026 usando Prophet

---

## Contenido:
1. Carga del dataset procesado
2. Preparación de datos para Prophet
3. Análisis de series temporales
4. Modelos de forecasting por canal
5. Evaluación de resultados
6. Visualización de predicciones 2026
7. Exportación de resultados

## 1. Configuración inicial y librerías

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from prophet import Prophet
from datetime import datetime, timedelta
import json

# Ignorar warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Librerías importadas correctamente")

## 2. Carga del dataset procesado

In [ ]:
# Definir rutas del proyecto
DATA_PROCESSED = Path('../data/processed')
DATA_FORECASTS = Path('../data/forecasts')
MODELS_DIR = Path('../models')
RESULTS_FIGURES = Path('../results/figures/forecasts')
RESULTS_REPORTS = Path('../results/reports')

# Crear carpetas si no existen
DATA_FORECASTS.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_FIGURES.mkdir(parents=True, exist_ok=True)
RESULTS_REPORTS.mkdir(parents=True, exist_ok=True)

# Cargar dataset limpio
dataset_file = DATA_PROCESSED / 'dataset_clean.csv'

if dataset_file.exists():
    df = pd.read_csv(dataset_file)
    print(f"✅ Dataset limpio cargado exitosamente")
    print(f"📊 Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")
else:
    print(f"❌ Error: No se encontró el archivo {dataset_file}")
    print(f"📁 Ejecuta primero el Notebook 02 para generar el dataset limpio")

In [ ]:
# Mostrar estructura del dataset
print("📋 Estructura del dataset:\n")
df.head(10)

In [ ]:
# Información del dataset
print("ℹ️ Información del dataset:\n")
print(f"Canales: {sorted(df['channel'].unique())}")
print(f"Período: {df['year'].min()}-{df['month'].min()} a {df['year'].max()}-{df['month'].max()}")
print(f"Total de meses: {df[['year', 'month']].drop_duplicates().shape[0]}")

## 3. Preparación de datos para Prophet

Prophet requiere un DataFrame con columnas específicas:
- `ds`: Fecha (datetime)
- `y`: Valor a predecir (numérico)

In [ ]:
# Crear columna de fecha
df['ds'] = pd.to_datetime(
    df['year'].astype(str) + '-' + df['month'].astype(str) + '-01'
)

print("✅ Columna de fecha 'ds' creada para Prophet")
print(f"\nRango de fechas: {df['ds'].min()} a {df['ds'].max()}")

In [ ]:
# ===================================================================
# CONFIGURACIÓN: Métricas y canales para forecasting
# ===================================================================

# Métricas a predecir (sin 'users' por ahora)
metricas_forecast = [
    'sessions',
    'bounce_rate',
    'views_per_session',
    'avg_session_duration'
]

# Canales a analizar (automático: todos los canales del dataset)
canales = sorted(df['channel'].unique())

# Horizonte de predicción: 12 meses (todo 2026)
periodos_forecast = 12

print(f"📊 Configuración de forecasting:")
print(f"\nMétricas a predecir: {len(metricas_forecast)}")
for metrica in metricas_forecast:
    print(f"   - {metrica}")

print(f"\nCanales a analizar: {len(canales)}")
for canal in canales:
    print(f"   - {canal}")

print(f"\nHorizonte de predicción: {periodos_forecast} meses (todo 2026)")
print(f"\nTotal de modelos a entrenar: {len(metricas_forecast)} métricas × {len(canales)} canales = {len(metricas_forecast) * len(canales)} modelos")

## 4. Análisis visual de series temporales

Antes de hacer forecasting, visualicemos las series temporales de cada métrica por canal.

In [ ]:
# Función para visualizar series temporales
def plot_series_temporales(df, metrica, titulo):
    """
    Grafica la evolución temporal de una métrica para todos los canales
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    for canal in sorted(df['channel'].unique()):
        data_canal = df[df['channel'] == canal].sort_values('ds')
        ax.plot(data_canal['ds'], data_canal[metrica], 
                marker='o', label=canal, linewidth=2, markersize=6)
    
    ax.set_xlabel('Fecha', fontsize=12)
    ax.set_ylabel(titulo, fontsize=12)
    ax.set_title(f'Evolución de {titulo} por Canal (2025)', fontsize=14, fontweight='bold')
    ax.legend(title='Canal', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Visualizar cada métrica
metricas_titulos = {
    'sessions': 'Sesiones',
    'bounce_rate': 'Bounce Rate (%)',
    'views_per_session': 'Vistas por Sesión',
    'avg_session_duration': 'Duración Promedio (segundos)'
}

for metrica, titulo in metricas_titulos.items():
    print(f"\n📈 Gráfico: {titulo}")
    plot_series_temporales(df, metrica, titulo)

## 5. Función de forecasting con Prophet

Crearemos una función para entrenar modelos Prophet y generar predicciones.

In [ ]:
def crear_forecast_prophet(df_canal, metrica, periodos=12):
    """
    Crea un modelo Prophet y genera predicciones.
    
    Args:
        df_canal (pd.DataFrame): DataFrame con columnas 'ds' y la métrica
        metrica (str): Nombre de la métrica a predecir
        periodos (int): Número de períodos (meses) a predecir
    
    Returns:
        tuple: (modelo, forecast_df, metricas_evaluacion)
    """
    # Preparar datos para Prophet
    df_prophet = df_canal[['ds', metrica]].rename(columns={metrica: 'y'})
    
    # Crear y entrenar modelo
    model = Prophet(
        yearly_seasonality=False,  # No tenemos datos de varios años
        weekly_seasonality=False,  # Datos mensuales
        daily_seasonality=False,   # Datos mensuales
        interval_width=0.95        # Intervalo de confianza del 95%
    )
    
    model.fit(df_prophet)
    
    # Crear dataframe futuro
    future = model.make_future_dataframe(periods=periodos, freq='MS')  # MS = Month Start
    
    # Generar predicciones
    forecast = model.predict(future)
    
    # Calcular métricas de evaluación en datos históricos
    # Comparar valores reales vs predichos en el período histórico
    forecast_historico = forecast[forecast['ds'].isin(df_prophet['ds'])]
    valores_reales = df_prophet['y'].values
    valores_predichos = forecast_historico['yhat'].values
    
    mae = np.mean(np.abs(valores_reales - valores_predichos))
    rmse = np.sqrt(np.mean((valores_reales - valores_predichos)**2))
    mape = np.mean(np.abs((valores_reales - valores_predichos) / valores_reales)) * 100
    
    metricas_eval = {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape
    }
    
    return model, forecast, metricas_eval

print("✅ Función de forecasting con Prophet creada")

## 6. Generación de forecasts por canal y métrica

Ahora entrenaremos modelos para cada combinación de canal + métrica.

In [ ]:
# Diccionario para almacenar todos los resultados
resultados_forecasts = {}
metricas_evaluacion = {}

print("🚀 Iniciando generación de forecasts...\n")
print("="*70)

total_modelos = len(canales) * len(metricas_forecast)
modelo_actual = 0

for canal in canales:
    print(f"\n📊 Canal: {canal}")
    print("-" * 70)
    
    # Filtrar datos del canal
    df_canal = df[df['channel'] == canal].sort_values('ds').reset_index(drop=True)
    
    resultados_forecasts[canal] = {}
    metricas_evaluacion[canal] = {}
    
    for metrica in metricas_forecast:
        modelo_actual += 1
        print(f"\n   [{modelo_actual}/{total_modelos}] Métrica: {metrica}")
        
        # Generar forecast
        model, forecast, metrics = crear_forecast_prophet(
            df_canal, 
            metrica, 
            periodos=periodos_forecast
        )
        
        # Guardar resultados
        resultados_forecasts[canal][metrica] = forecast
        metricas_evaluacion[canal][metrica] = metrics
        
        # Mostrar métricas de evaluación
        print(f"      MAE:  {metrics['MAE']:.2f}")
        print(f"      RMSE: {metrics['RMSE']:.2f}")
        print(f"      MAPE: {metrics['MAPE']:.2f}%")

print("\n" + "="*70)
print("✅ Todos los modelos entrenados exitosamente")
print(f"\nTotal de modelos: {total_modelos}")

## 7. Resumen de métricas de evaluación

In [ ]:
# Crear DataFrame con resumen de métricas de evaluación
resumen_evaluacion = []

for canal in canales:
    for metrica in metricas_forecast:
        metrics = metricas_evaluacion[canal][metrica]
        resumen_evaluacion.append({
            'Canal': canal,
            'Métrica': metrica,
            'MAE': metrics['MAE'],
            'RMSE': metrics['RMSE'],
            'MAPE (%)': metrics['MAPE']
        })

df_evaluacion = pd.DataFrame(resumen_evaluacion)

print("📊 Resumen de métricas de evaluación de modelos:\n")
display(df_evaluacion)

In [ ]:
# Guardar resumen de evaluación
eval_file = RESULTS_REPORTS / 'model_evaluation_metrics.csv'
df_evaluacion.to_csv(eval_file, index=False)
print(f"\n💾 Métricas de evaluación guardadas en: {eval_file}")

## 8. Visualización de predicciones 2026

Visualizaremos las predicciones para cada métrica y canal.

In [ ]:
def plot_forecast(df_historico, forecast, canal, metrica, titulo):
    """
    Grafica datos históricos y predicciones con intervalos de confianza
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Datos históricos
    ax.plot(df_historico['ds'], df_historico[metrica], 
            'o-', color='black', label='Histórico', linewidth=2, markersize=6)
    
    # Predicciones
    forecast_futuro = forecast[forecast['ds'] > df_historico['ds'].max()]
    ax.plot(forecast_futuro['ds'], forecast_futuro['yhat'], 
            'o-', color='#0072B2', label='Predicción', linewidth=2, markersize=6)
    
    # Intervalo de confianza
    ax.fill_between(forecast_futuro['ds'], 
                     forecast_futuro['yhat_lower'], 
                     forecast_futuro['yhat_upper'],
                     alpha=0.3, color='#0072B2', label='Intervalo de confianza (95%)')
    
    ax.set_xlabel('Fecha', fontsize=12)
    ax.set_ylabel(titulo, fontsize=12)
    ax.set_title(f'{titulo} - {canal}', fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Guardar gráfico
    filename = f"forecast_{metrica}_{canal.replace(' ', '_').lower()}.png"
    plt.savefig(RESULTS_FIGURES / filename, dpi=300, bbox_inches='tight')
    plt.show()
    
    return filename

print("✅ Función de visualización creada")

In [ ]:
# Generar gráficos para todas las combinaciones
print("📊 Generando visualizaciones de predicciones...\n")

graficos_generados = []

for canal in canales:
    print(f"\n📈 Canal: {canal}")
    print("-" * 70)
    
    df_canal = df[df['channel'] == canal].sort_values('ds')
    
    for metrica in metricas_forecast:
        print(f"   Gráfico: {metrica}")
        
        forecast = resultados_forecasts[canal][metrica]
        titulo = metricas_titulos[metrica]
        
        filename = plot_forecast(df_canal, forecast, canal, metrica, titulo)
        graficos_generados.append(filename)

print(f"\n✅ {len(graficos_generados)} gráficos generados y guardados en: {RESULTS_FIGURES}")

## 9. Exportación de predicciones 2026

Consolidaremos todas las predicciones en un único dataset.

In [ ]:
# Extraer solo las predicciones de 2026
predicciones_2026 = []

for canal in canales:
    for metrica in metricas_forecast:
        forecast = resultados_forecasts[canal][metrica]
        
        # Filtrar solo fechas de 2026
        forecast_2026 = forecast[forecast['ds'].dt.year == 2026][['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
        
        # Agregar información del canal y métrica
        forecast_2026['channel'] = canal
        forecast_2026['metric'] = metrica
        forecast_2026['year'] = 2026
        forecast_2026['month'] = forecast_2026['ds'].dt.month
        
        # Renombrar columnas
        forecast_2026 = forecast_2026.rename(columns={
            'yhat': 'predicted_value',
            'yhat_lower': 'lower_bound',
            'yhat_upper': 'upper_bound'
        })
        
        predicciones_2026.append(forecast_2026)

# Consolidar en un DataFrame
df_predicciones = pd.concat(predicciones_2026, ignore_index=True)

# Reordenar columnas
df_predicciones = df_predicciones[[
    'year', 'month', 'channel', 'metric', 
    'predicted_value', 'lower_bound', 'upper_bound', 'ds'
]].sort_values(['channel', 'metric', 'year', 'month']).reset_index(drop=True)

print("📊 Predicciones 2026 consolidadas:\n")
print(f"Total de predicciones: {len(df_predicciones)}")
print(f"\nPrimeras filas:")
display(df_predicciones.head(15))

In [ ]:
# Guardar predicciones en CSV
forecast_file = DATA_FORECASTS / 'forecasts_2026_all_channels.csv'
df_predicciones.to_csv(forecast_file, index=False)

print(f"✅ Predicciones guardadas en: {forecast_file}")
print(f"📊 Dimensiones: {df_predicciones.shape[0]} filas x {df_predicciones.shape[1]} columnas")
print(f"💾 Tamaño: {forecast_file.stat().st_size / 1024:.2f} KB")

## 10. Resumen ejecutivo de predicciones

In [ ]:
# Crear resumen por canal y métrica (promedio anual 2026)
resumen_2026 = df_predicciones.groupby(['channel', 'metric']).agg({
    'predicted_value': ['mean', 'sum', 'min', 'max'],
    'lower_bound': 'mean',
    'upper_bound': 'mean'
}).round(2)

print("📊 Resumen de predicciones 2026 por canal y métrica:\n")
display(resumen_2026)

In [ ]:
# Guardar resumen ejecutivo
resumen_file = RESULTS_REPORTS / 'summary_forecasts_2026.csv'
resumen_2026.to_csv(resumen_file)
print(f"\n💾 Resumen ejecutivo guardado en: {resumen_file}")

In [ ]:
# Resumen final
print("="*70)
print("🎯 RESUMEN FINAL DEL FORECASTING")
print("="*70)

print(f"\n1. MODELOS ENTRENADOS:")
print(f"   - Total de modelos: {len(canales) * len(metricas_forecast)}")
print(f"   - Canales analizados: {len(canales)}")
print(f"   - Métricas predichas: {len(metricas_forecast)}")

print(f"\n2. PERÍODO DE PREDICCIÓN:")
print(f"   - Horizonte: 12 meses (2026)")
print(f"   - Desde: Enero 2026")
print(f"   - Hasta: Diciembre 2026")

print(f"\n3. ARCHIVOS GENERADOS:")
print(f"   - Predicciones completas: {forecast_file.name}")
print(f"   - Resumen ejecutivo: {resumen_file.name}")
print(f"   - Métricas de evaluación: {eval_file.name}")
print(f"   - Gráficos: {len(graficos_generados)} archivos PNG")

print(f"\n4. MÉTRICAS PREDICHAS:")
for metrica in metricas_forecast:
    print(f"   - {metrica}")

print(f"\n5. PRÓXIMO PASO:")
print(f"   ✓ Notebook 04: Visualización y análisis de resultados")
print(f"   ✓ Generación de reportes ejecutivos")
print(f"   ✓ Exportación de dashboards")

print("\n" + "="*70)

---

## Notas importantes:

### Sobre Prophet:
- ✅ Modelo automático de Facebook optimizado para series temporales
- ✅ Genera intervalos de confianza del 95%
- ✅ Robusto con datos faltantes y outliers
- ⚠️ Con solo 11 meses de histórico, la precisión puede ser limitada

### Sobre las predicciones:
- Las predicciones deben interpretarse como **estimaciones** basadas en tendencias históricas
- Los intervalos de confianza indican el rango probable de valores
- Eventos externos (cambios de algoritmo, estacionalidad no capturada) pueden afectar la precisión

### Métricas de evaluación:
- **MAE** (Mean Absolute Error): Error promedio absoluto
- **RMSE** (Root Mean Squared Error): Error cuadrático medio
- **MAPE** (Mean Absolute Percentage Error): Error porcentual promedio

## ✅ Archivos generados:

- `data/forecasts/forecasts_2026_all_channels.csv` - Predicciones completas
- `results/reports/summary_forecasts_2026.csv` - Resumen ejecutivo
- `results/reports/model_evaluation_metrics.csv` - Métricas de evaluación
- `results/figures/forecasts/*.png` - Gráficos de predicciones

## Próximo paso:

**Notebook 04:** Visualización avanzada y reportes finales